In [ ]:
#!/usr/bin/env


"""


split, clean, and self-cal continuum and line data
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

III_79    II   M4.0
16:09:01.400 -39:25:11.899


In [ ]:
field = 34                                     # CHANGEME 
file_ms = '../science_calibrated.ms'
contspw   = '2,3,4,7,8,9'                      # continuum spectral windows
contspw_w = [128,3840,1920,128,3840,1920]      # continuum spw widths
robust     = 0.5                               # CHANGEME
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
refant     = 'DA52'                            # CHANGEME
xc         = 318                               # CHANGEME
yc         = 307                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 1.25
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Split Off Continuum 

split off field from full ms


In [ ]:
split(vis = file_ms,
      outputvis = 'f'+str(field)+'.vis',
      field = field,
      datacolumn = 'data')


split off continuum (take the large bw spw and average


In [ ]:
split(vis = 'f'+str(field)+'.vis',
      outputvis = 'f'+str(field)+'_cont.vis',
      spw = contspw,
      width = contspw_w,
      datacolumn = 'data')


plot uv-distance vs. amplitude


In [ ]:
plotms(vis='f'+str(field)+'_cont.vis',
       xaxis='uvdist',yaxis='amp',
       coloraxis='spw')
       # plotfile='f'+str(field)+'_ampuv_orig.png'
       # showgui=False,
       # highres=True,
       # overwrite=True)


source is unresolved
find antenna close to center of configuration
check pipeline log that this ant is OK


In [ ]:
plotants(vis='f'+str(field)+'_cont.vis') #, figfile='f'+str(field)+'_ants.png')


#  Clean continuum before selfcal 

light clean (100 iterations) to set the mask around the main peaks
os.system('rm -rf f'+str(field)+'_cont_b4sc*')


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_b4sc',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 1000,
      threshold = '0.0mJy',
      interactive = True,
      mask = '',
      cell = cell,
      imsize = imsize,
      weighting = 'briggs',
      robust = robust,
      imagermode = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_b4sc.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_b4sc.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)


Peak = 4.87 mJy, rms = 0.23 mJy, S/N = 21.6


#  Self-Calibrate 1 

first combine all the data by time (solint = inf)
i.e., phase self-cal over entire integration time


In [ ]:
gaincal(vis = 'f'+str(field)+'_cont.vis',
        caltable = 'f'+str(field)+'_cont_pcal1',
        refant = refant,
        solint = 'inf',
        combine = 'spw',
        gaintype = 'T',
        spw = '',
        calmode = 'p',
        minblperant = 4,
        minsnr = 3)


6 of 32 solutions flagged due to SNR < 3 in spw=0 at 2015/06/15/02:21:26.7
4 of 31 solutions flagged due to SNR < 3 in spw=0 at 2015/06/15/04:33:19.5
plot phase for each antenna


In [ ]:
plotcal(caltable = 'f'+str(field)+'_cont_pcal1',
        xaxis = 'time',
        yaxis = 'phase',
        spw = '',
        iteration = 'antenna',
        subplot = 421,
        plotrange = [0,0,-200,200]) 


apply calibration to data


In [ ]:
applycal(vis = 'f'+str(field)+'_cont.vis',
        spw = '',
        gaintable = ['f'+str(field)+'_cont_pcal1'],
        spwmap = [0,0,0,0,0,0],
        calwt = T,
        flagbackup = F)


clean self-calibrated data


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_pcal1_clean',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 100,
      threshold   = '0.0mJy',
      interactive = False,
      mask = 'f'+str(field)+'_cont_b4sc.mask',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = robust,
      imagermode  = imagermode)
im_max = imstat(imagename = 'f'+str(field)+'_cont_pcal1_clean.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_pcal1_clean.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)


Peak = 6.53 mJy, rms = 0.28 mJy, S/N = 23.2
inspect images


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cont_b4sc.image'},
               {'file':'f'+str(field)+'_cont_pcal1_clean.image'}])


pcal1 significantly worse


#  Self-Calibrate 2 

decrease phase self-cal solution interval to a few times integration time
int = 6s (from X125.log)


In [ ]:
gaincal(vis = 'f'+str(field)+'_cont.vis',
        caltable = 'f'+str(field)+'_cont_pcal2',
        refant = refant,
        solint = '20s',                           # CHANGEME
        combine = 'spw',
        gaintype = 'T',
        spw = '',
        calmode = 'p',
        minblperant = 4,
        minsnr = 3)


#TOO MANY FLAGS, STOPPING HERE


#  Best Continuum Map 

do not apply self-cal. all were horrible, due to faint source?
RE-CREATE _CONT.VIS FILE!!!!
deep clean, trying different robust weights
os.system('rm -rf f'+str(field)+'_cont_best*')


In [ ]:
clean(vis = 'f'+str(field)+'_cont.vis',
      imagename = 'f'+str(field)+'_cont_best',
      mode = 'mfs',
      psfmode = 'clark',
      niter = 2000,
      threshold   = '0.0mJy',
      interactive = True,
      mask = '',
      cell        = cell,
      imsize      = imsize,
      weighting   = 'briggs',
      robust      = -1.0,                                      # CHANGEME
      imagermode  = imagermode)


placed mask around outer cont contour
stopped after 300 iterations once the inside became green


In [ ]:
im_max = imstat(imagename = 'f'+str(field)+'_cont_best.image')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
bmaj = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beammajor")
bmin = imhead(imagename = 'f'+str(field)+'_cont_best.image', mode="get", hdkey="beamminor")
print 'Peak = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_max, 1000*im_rms, im_max/im_rms)
print 'Beam = {0:.2f} x {1:.2f} arcsec'.format(bmaj.get('value'),bmin.get('value'))


robust = -1.0
Peak = 4.46 mJy, rms = 0.29 mJy, S/N = 15.2
Beam = 0.32 x 0.26 arcsec
save this to a fits file


In [ ]:
exportfits(imagename='f'+str(field)+'_cont_best.image', fitsimage='f'+str(field)+'_cont.fits')


measure flux
imview(raster=[{'file':'f'+str(field)+'_cont_best.image'}])


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cont_best.image',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 15.56 mJy, rms = 0.30 mJy, S/N = 52.5
re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_cont_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cont_cropped.image',
             infile  = 'f'+str(field)+'_cont.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cont_cropped.image',
           fitsimage = 'f'+str(field)+'_cont_cropped.fits')


In [ ]:
Measuring COG for M/f34_cont_cropped.fits
Assuming object center (300.0,300.0)
Background: -0.00 mJy/beam km/s
RMS in annulus 4.0-9.0 arcsec = 0.30 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10     1.30     0.08     17.2
   1     0.20     4.32     0.22     19.5
   2     0.30     8.30     0.28     29.6
   3     0.40    11.89     0.38     31.1
   4     0.50    14.90     0.51     29.3
   5     0.60    16.64     0.59     28.4
   6     0.70    17.39     0.64     27.3
   7     0.80    17.50     0.68     25.6
   8     0.90    17.22     0.93     18.4
   9     1.00    16.77     0.96     17.5
  10     1.10    16.35     1.04     15.8
  11     1.20    15.72     0.95     16.6
  12     1.30    15.59     1.17     13.3
  13     1.40    16.13     1.24     13.0
  14     1.50    16.76     1.29     13.0
F = 17.50 mJy
E = 0.68 mJy
S = 25.59
D = 1.60 arcsec


#  Measure flux with UVMODELFIT 

calculate offset from phase center in arcsec


In [ ]:
pixscale = 0.03             # must match 'cell'                 
dx = pixscale*(320.0-xc)    # offset to east (left)
dy = pixscale*(yc-320.0)    # offset to north (up)


measure flux as gaussian


In [ ]:
uvmodelfit(vis       = 'f'+str(field)+'_cont.vis',
           comptype  = 'G',
           sourcepar = [im_flux,dx,dy,0.5,1,0],
           varypar   = [T,T,T,T,T,T],
           niter     = 10)


In [ ]:
I = 0.0183079 +/- 0.000571586
x = 0.0591895 +/- 0.00477235 arcsec
y = -0.445259 +/- 0.010732 arcsec
a = 0.787902 +/- 0.0267875 arcsec
r = 0.289697 +/- 0.0230327
p = -2.86211 +/- 1.27516 deg
consistent with aperture methods
16:09:01.405 -39:25:12.345
